# Исследование надежности заемщиков


**Цели и задачи проекта:** На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.

## Откроем таблицу и изучим общую информацию о данных

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('c:/users/user/Desktop/datasets/data.csv')

**Выведим первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведим основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведим количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце мы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Обработаем значения: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведим медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

**Выведим перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведим перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведим количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Выведим на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создаем в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведим на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создаем функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Посмотрим, есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_pivot_children = data.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot_children['debt_share'] = data_pivot_children['sum'] / data_pivot_children['count'] * 100
data_pivot_children['debt_share'] = round(data_pivot_children['debt_share'], 2)
data_pivot_children['debt_share'] = data_pivot_children['debt_share'].astype(str)
data_pivot_children['debt_share'] = data_pivot_children['debt_share'] + '%'

def categorize_children(children):
    try:
        if  children == 0:
            return 'Без детей'
        elif children >= 1:
            return 'С детьми'
    except:
        pass
data['children_category'] = data['children'].apply(categorize_children)
data_pivot_children = data.pivot_table(index = 'children_category', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot_children['debt_share'] = data_pivot_children['sum'] / data_pivot_children['count'] * 100
data_pivot_children['debt_share'] = round(data_pivot_children['debt_share'], 2)
data_pivot_children['debt_share'] = data_pivot_children['debt_share'].astype(str)
data_pivot_children['debt_share'] = data_pivot_children['debt_share'] + '%'
display(data_pivot_children)

,count,sum,debt_share
,debt,debt,
children_category,,,
Без детей,14091,1063,7.54%
С детьми,7240,669,9.24%


**Вывод:** 

Из полученных данных можно сделать вывод, что заемщики без детей пользуются кредитными средствами почти в 2 раза больше чем заемщики с детьми. В виду того, что группы заемщиков с количеством детей равным 3-м и более малочисленны (*в сумме составляет менее 2% от общего количества заемщиков*) и не репрезентативны, а показатели групп заемщиков с детьми в количестве 1 и 2 отличаются незначительно, заемщики поделены на 2 основные категории: "Без детей" и "С детьми".

**Наблюдается следующая зависимость между количеством детей и возвратом кредита в срок:**
1. Самыми надежными заемщиками по критерию возврата кредита в срок являются заемщики без детей (*самая больщая группа заемщиков*). Доля "невозврата" в данной группе составляет 7.54%.
2. На втором месте - заемщики с детьми. Доля "невозврата" в данной группе превышает долю "невозврата" группы заемщиков без детей на 1.7%.

#### 3.2 Посмотрим, есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_pivot_family = data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot_family['debt_share'] = data_pivot_family['sum'] / data_pivot_family['count'] * 100
data_pivot_family['debt_share'] = round(data_pivot_family['debt_share'], 2)
data_pivot_family['debt_share'] = data_pivot_family['debt_share'].astype(str)
data_pivot_family['debt_share'] = data_pivot_family['debt_share'] + '%'
display(data_pivot_family)

,count,sum,debt_share
,debt,debt,
family_status,,,
Не женат / не замужем,2796,273,9.76%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%


**Вывод:** Самой большой по численности категорией заемщиков явялются заемщики, состоящие в браке. Их количество превышает количество заемщиков других категорий минимум в 3 раза.

**Наблюдается следующая зависимость между семейным положением и возвратом кредита в срок:**
1. Самыми надежными по критерию возврата кредита в срок являются "вдовцы/водвы" (*самая маленькая группа заемщиков*). Доля "невозврата" в данной группе составляет 6.62%.
2. На втором месте по надежности по критерию возврата кредита в срок являются заемщики, находящиеся в группах "женат/замужем" (*самая большая группа заемщиков*) и "в разводе". Данные группы демонтрируют схожие показатели по "невозврату": доля "невозврата" в данных группах в среднем выше на 0,69% показателей самой надежной группы "вдовцы/вдовы".
3. На третьем месте с самой высокая долей "невозврата" - заемщики, относиещся к группам "не женат/не замужем" и "гражданский брак". Доля "невозврата" в данных группах в среднем выше на 2.91% показателей самой надежной группы "вдовцы/вдовы".

#### 3.3 Посмотрим, есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data_pivot_total_income = data.pivot_table(index = 'total_income_category', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot_total_income['share'] = data_pivot_total_income['sum']/data_pivot_total_income['count']*100
data_pivot_total_income['share'] = round(data_pivot_total_income['share'], 2)
data_pivot_total_income['share'] = data_pivot_total_income['share'].astype(str)
data_pivot_total_income['share'] = data_pivot_total_income['share'] + '%'
display(data_pivot_total_income)

,count,sum,share
,debt,debt,
total_income_category,,,
A,25,2,8.0%
B,5014,354,7.06%
C,15921,1353,8.5%
D,349,21,6.02%
E,22,2,9.09%


**Вывод:** 
1. Данные указывают, что самыми надежными заемщиками по критерию возврата кредита в срок являются заемщики с уровнем дохода категории D (от 30001 до 50000). Доля "невозврата" в данной категории составляет 6,02%.
2. На втором месте по данному критерию находятся заемщики с уровнем дохода категории В (*от 200001 до 1000000*). Уровень "невозврата" в данной категории превышает долю "невозврата" категории D на 1,04%.
3. На третьем месте по данному критерию находятся заемщики с уровнем дохода А (*самый высокий уровень, более 1000001*) и С (*от 50001 до 200000*)(*С - самая большая группа заемщиков*), которые проявляют примерно одинаковый уровень возврата кредитов в срок. В среднем, доля "невозврата" данной категории превышает долю "невозврата" категории D на 2.23%. 
4. Самый высокий уровень "невозврата" наблюдается у самой малочисленной группы заемщиков с уровнем дохода категории Е (*менее 30000*). Доля "невозврата" данной категории превышает долю "невозврата" категории D на 3.07%.


#### 3.4 Посмотрим, как разные цели кредита влияют на его возврат в срок?

In [25]:
data_pivot_purpose = data.pivot_table(index = 'purpose_category', values = 'debt', aggfunc = ['count', 'sum'])
data_pivot_purpose['share_of_debt'] = data_pivot_purpose['sum'] / data_pivot_purpose['count'] * 100
data_pivot_purpose['share_of_debt'] = round(data_pivot_purpose['share_of_debt'], 2)
data_pivot_purpose['share_of_debt'] = data_pivot_purpose['share_of_debt'].astype(str)
data_pivot_purpose['share_of_debt'] = data_pivot_purpose['share_of_debt'] + '%'
display(data_pivot_purpose)

,count,sum,share_of_debt
,debt,debt,
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


**Вывод:** 
1. Наиболее надежными по критерию возврата кредита в срок являются займы, выданные на "операции с недвижимостью" (*самая большая группа заемщиков*). Доля "невозврата" в данной категории составляет 7,26%.
2. На втором месте с небольшой разницей по данному критерию находятся займы, выданные на "проведение свадьбы". Доля "невозварата" в данной категории превышает долю "невозврата" в категории "операции с недвижимостью" на 0,65%.
3. На третьем месте по данному критерию находятся займы, выданные на "операции с автомобилем" и "получение образования". В среднемдоля "невозврата" в данных категориях превышает долю "невозврата" в катеригории "операции с недвижимостью" на 2.04%.

#### 3.5 Возможные причины появления пропусков в исходных данных.


**Причинами появления пропусков могут служить:**
1. Человечкий фактор: данные не были предоставлены самими заемщиками либо данные не были внесены оператором баз данных.
2. Человеческий фактор: учитывая специфику данных датафрейма, возможно в местах появления пропусков заемщиками были указаны сведения с 0 значенииями, но для дальнешего успешного оформления займа данные были удалены специально, чтобы они отрицательно не повлияли на принятие решений по выдаче займа. 
3. Технологические ошибки, которые возникли при копировании, считывании, смене форматов или при другой технолгической ошибке, которые привели к удалению или несохранению данных.

### Шаг 4: общий вывод.

**Общий вывод**

По результатам исследования выяснили, что существует зависимость возрвата кредита в срок от таких показателей как количество детей у заемщиков, семейное положение, уровень дохода, цели кредита. При этом зависимость нелинейная. Например, самый высокий уровень дохода не гарантирует самые лучшие показатели по возврату кредита в срок. Каждая группа в данных категориях заемщиков демонстрирует собственные показатели.

В зависимости от целей проведения данного исследования, рекомендуется обратить внимание на группы заемщиков в данных категориях, которые являются самыми большими по численности, так как их показатели в абсолютном денежном выражении могут существенно повлиять на выручку. (*Для подтверждения данной гипотезы рекомендуется провести дополнительное исследование с учетом объемов кредитования в денежном выражении.*)

Также рекомендуется обратить пристальное внимание на объемы кредитования. Так, например, объемы кредитования для групп с уровнем дохода категории А могут быть значительно выше и финансовые потери от "невозврата" или "несовевременного возврата" в абсолютном денежном выражении выше чем в случаях с заемщиками других категорий.(*Для подтверждения данной гипотезы рекомендуется провести дополнительное исследование.*)

В исходных были пропуски в двух колонках "days_employed" и "total_incomе", количество пропусков одинаковое. Из этого можно сделать предположение, что пропуски имеют неслучайный характер. Данные из двух таблиц обычно ялвяются обязательными сведениями для оформления кредита, так как показывают финансовую состоятельность заемщика. Рекомендуется провести исследование и выяснить причинно-следственную связь пропусков, так как возможны нарушения регламента принятия решения по кредитованию, неправомерные действия со стороны сотрудников и/или заемщиков, несоотвесттвие банковской требований актуальным требованиям ЦБ по вопросам кредитования, технологические ошибки в системе и прочее.
